# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click !here goes the icon of the corresponding button in the gutter! button.
To debug a cell, press Alt+Shift+Enter, or click !here goes the icon of the corresponding button in the gutter! button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/jupyter-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [11]:
import time
import os
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed

# Initialize the WebDriver options (for better performance)
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode for speed
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Function to get all company codes from the dropdown list
def get_company_codes(driver):
    url = "https://www.mse.mk/mk/stats/symbolhistory/ADIN"
    driver.get(url)
    time.sleep(3)
    
    dropdown = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, "Code"))
    )
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    options = soup.select('#Code option')
    company_codes = [option['value'] for option in options if option['value'] and not re.search(r'\d', option['value'])]
    
    print(f"Retrieved {len(company_codes)} company codes.")
    return company_codes

# Function to get the latest date available in the CSV file for each company
def get_latest_date_for_company(code):
    csv_file = f"{code}.csv"
    if not os.path.exists(csv_file):
        print(f"No data found for {code}")
        return None
    
    df = pd.read_csv(csv_file)
    if df.empty:
        print(f"CSV EMPTY {code}")
        return None
    
    latest_date = pd.to_datetime(df['Датум'], dayfirst=True).max()
    print(f"{latest_date.strftime('%d.%m.%Y')} for company {code}")
    return latest_date

# Function to update company data
def update_company_data(code, start_date, driver):
    end_date = datetime.today()
    if start_date > end_date:
        print(f"Start date {start_date} is greater than today's date. Adjusting start date to {end_date}.")
        start_date = end_date  # Adjust start_date to today if it's greater than end_date

    current_date = start_date
    print(f"Scraping data for company: {code}")

    # Loop until current_date exceeds end_date
    while current_date <= end_date:
        days_diff = (end_date - current_date).days

        if days_diff <= 7:
            to_date = current_date + timedelta(days=days_diff)
        elif days_diff <= 30:
            to_date = current_date + timedelta(days=7)  # Fetch weekly if under a month gap
        elif days_diff <= 365:
            to_date = current_date + timedelta(days=30)  # Fetch monthly if within a year
        else:
            to_date = current_date + timedelta(days=365)  # Fetch yearly if over a year

        # Ensure 'from_date' is always earlier than or equal to 'to_date'
        if to_date <= current_date:
            to_date = current_date

        # Format dates for request
        from_date_str = current_date.strftime('%d.%m.%Y')
        to_date_str = to_date.strftime('%d.%m.%Y')

        url = f"https://www.mse.mk/mk/stats/symbolhistory/{code}"
        driver.get(url)

        try:
            from_date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "FromDate")))
            to_date_input = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "ToDate")))

            from_date_input.clear()
            from_date_input.send_keys(from_date_str)
            to_date_input.clear()
            to_date_input.send_keys(to_date_str)

            show_button = driver.find_element(By.CSS_SELECTOR, 'input[value="Прикажи"]')
            show_button.click()

            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#resultsTable")))

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            table = soup.select_one('#resultsTable tbody')

            rows = [
                [code] + [col.get_text(strip=True) if col.get_text(strip=True) != '0' else None for col in row.select('td')]
                for row in table.select('tr') if row.select('td')
            ]

            if rows:
                new_data = pd.DataFrame(rows, columns=["Код", "Датум", "Цена на последна трансакција", "Мак.", "Мин.", "Просечна цена", "%пром.", "Количина", "Промет во БЕСТ во денари", "Вкупен промет во денари"])
                new_data['Датум'] = pd.to_datetime(new_data['Датум'], dayfirst=True)

                csv_file = f"{code}.csv"
                if os.path.exists(csv_file):
                    existing_data = pd.read_csv(csv_file, parse_dates=["Датум"], dayfirst=True)
                    combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset="Датум").sort_values(by="Датум")
                    combined_data.to_csv(csv_file, index=False, date_format="%d.%m.%Y")
                else:
                    new_data.to_csv(csv_file, index=False, date_format="%d.%m.%Y")

                print(f"Data updated for {code} from {from_date_str} to {to_date_str}")

        except TimeoutException:
            print(f"Timeout occurred for {code} while fetching data from {from_date_str} to {to_date_str}.")
            
            if to_date.year == datetime.today().year:
                to_date = datetime.today()

            # Move forward by one year
            else:
                to_date = current_date + timedelta(days=364)
            print(f"Moving forward with the next available range: {from_date_str} to {to_date.strftime('%d.%m.%Y')}")

        current_date = to_date + timedelta(days=1)

        if current_date > end_date:
            print(f"Finished processing {code}. Reached the end date: {end_date.strftime('%d.%m.%Y')}")
            break

# Main function to combine all operations
def main():
    # Start timer
    start_time = time.time()

    driver = webdriver.Chrome(options=options)

    # Get company codes
    all_codes = get_company_codes(driver)

    for code in all_codes:
        latest_date = get_latest_date_for_company(code)
        if latest_date:
            start_date = latest_date + timedelta(days=1)
        else:
            start_date = datetime.today() - timedelta(days=365 * 10)

        update_company_data(code, start_date, driver)

    # End timer
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Total execution time: {execution_time:.2f} seconds")

    driver.quit()

if __name__ == "__main__":
    main()


Retrieved 164 company codes.
08.11.2024 for company ADIN
Scraping data for company: ADIN
Timeout occurred for ADIN while fetching data from 09.11.2024 to 10.11.2024.
Moving forward with the next available range: 09.11.2024 to 10.11.2024
Finished processing ADIN. Reached the end date: 10.11.2024
08.11.2024 for company ALK
Scraping data for company: ALK
Timeout occurred for ALK while fetching data from 09.11.2024 to 10.11.2024.
Moving forward with the next available range: 09.11.2024 to 10.11.2024
Finished processing ALK. Reached the end date: 10.11.2024
28.12.2023 for company ALKB
Scraping data for company: ALKB
Timeout occurred for ALKB while fetching data from 29.12.2023 to 28.01.2024.
Moving forward with the next available range: 29.12.2023 to 10.11.2024
Finished processing ALKB. Reached the end date: 10.11.2024
08.11.2024 for company AMEH
Scraping data for company: AMEH
Timeout occurred for AMEH while fetching data from 09.11.2024 to 10.11.2024.
Moving forward with the next availabl

C:\Users\Leonora\AppData\Local\Temp\ipykernel_16440\602023161.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset="Датум").sort_values(by="Датум")


Data updated for CDHV from 15.11.2017 to 15.11.2018


C:\Users\Leonora\AppData\Local\Temp\ipykernel_16440\602023161.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset="Датум").sort_values(by="Датум")


Data updated for CDHV from 16.11.2018 to 16.11.2019


...

Data updated for ZUAS from 17.11.2019 to 16.11.2020
Timeout occurred for ZUAS while fetching data from 17.11.2020 to 17.11.2021.
Moving forward with the next available range: 17.11.2020 to 16.11.2021


C:\Users\Leonora\AppData\Local\Temp\ipykernel_16440\602023161.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset="Датум").sort_values(by="Датум")


Data updated for ZUAS from 17.11.2021 to 17.11.2022


C:\Users\Leonora\AppData\Local\Temp\ipykernel_16440\602023161.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([existing_data, new_data]).drop_duplicates(subset="Датум").sort_values(by="Датум")


Data updated for ZUAS from 18.11.2022 to 18.11.2023
Timeout occurred for ZUAS while fetching data from 19.11.2023 to 19.12.2023.
Moving forward with the next available range: 19.11.2023 to 17.11.2024
Finished processing ZUAS. Reached the end date: 10.11.2024
Total execution time: 5676.22 seconds
